Set up env

In [1]:
import carla
import numpy as np
from src.simulator import Simulator
from src.encoder import EncoderResnet18
import torch
from src.agent import NCPAgent
from src.model import Model, SequenceLearner

simulator = Simulator(world_name='Town04_opt')


In [2]:
simulator.world.get_spectator().set_transform(
    carla.Transform(
        location=carla.Location(x=398.7934265136719,
                                y=-56.03200912475586,
                                z=3.37939715385437)))

simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9) # camera coords
)
vehicle = simulator.get_vehicle()


output_size = 2
units = 512

ncp = Model(output_size, units)

agent = NCPAgent(simulator, ncp)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 20
for _ in range(150):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest = waypoints[6].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)

alloc!
Camera stream started


In [3]:
seq_learner = SequenceLearner(ncp, 0.005)

In [5]:
import torchvision
from IPython.display import display
import time
import sys
import pytorch_lightning as pl
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent

idx = 1
waypoint = waypoints[idx]
to_PIL = torchvision.transforms.ToPILImage()

basic_agent = BasicAgent(vehicle=vehicle)

trainer = pl.Trainer(
    logger=pl.loggers.CSVLogger("log"),
    max_epochs=5,
    gradient_clip_val=1, #clip grad for training stabilizing
)

while True:
    if vehicle.get_location().distance(dest) < 0.5:
        print(f"Destination reached")
        break
    if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
        (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
         vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
        print(f'Waypoint {idx} was reached')
        waypoint = waypoints[idx + 1]
        idx += 1
    
    control, movement, raw_data = agent.run_step()
    trainer.fit(seq_learner)
    print(control.steer)
    vehicle.apply_control(control)
    
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/hard_drive/habkaffee_part/programming/coursework/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: log/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type  | Params
--------------------------------
0 | model | Model | 2.6 M 
--------------------------------
2.1 M     Trainable params
524 K     Non-trainable par

MisconfigurationException: `train_dataloader` must be implemented to be used with the Lightning Trainer

In [6]:
simulator.destroy_all()

Camera stream stopped


In [ ]:
# # we will save the conv layer weights in this list
# model_weights =[]
# #we will save the 49 conv layers in this list
# conv_layers = []
# # get all the model children as list
# model_children = list(model.children())
# #counter to keep count of the conv layers
# counter = 0
# #append all the conv layers and their respective wights to the list
# for i in range(len(model_children)):
#     if type(model_children[i]) == nn.Conv2d:
#         counter+=1
#         model_weights.append(model_children[i].weight)
#         conv_layers.append(model_children[i])
#     elif type(model_children[i]) == nn.Sequential:
#         for j in range(len(model_children[i])):
#             for child in model_children[i][j].children():
#                 if type(child) == nn.Conv2d:
#                     counter+=1
#                     model_weights.append(child.weight)
#                     conv_layers.append(child)
# print(f"Total convolution layers: {counter}")
# print(f"{conv_layers}")

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)

In [ ]:
# image = transform(image)
# image = image[:3,:,:]
# print(f"Image shape before: {image.shape}")
# image = image.unsqueeze(0)
# print(f"Image shape after: {image.shape}")
# image = image.to(device)

In [ ]:
# outputs = []
# names = []
# for layer in conv_layers[0:]:
#     image = layer(image)
#     outputs.append(image)
#     names.append(str(layer))
# print(len(outputs))
# # print feature_maps
# for feature_map in outputs:
#     print(feature_map.shape)

In [ ]:
# processed = []
# for feature_map in outputs:
#     feature_map = feature_map.squeeze(0)
#     gray_scale = torch.sum(feature_map,0)
#     gray_scale = gray_scale / feature_map.shape[0]
#     processed.append(gray_scale.data.cpu().numpy())
# for fm in processed:
#     print(fm.shape)


In [ ]:
# fig = plt.figure(figsize=(30, 50))
# for i in range(len(processed)):
#     a = fig.add_subplot(5, 4, i+1)
#     imgplot = plt.imshow(processed[i])
#     a.axis("off")
#     a.set_title(names[i].split('(')[0], fontsize=30)
# plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')


In [ ]:
# import torch
# import torch.nn as nn
# import pytorch_lightning as pl
# import torch.utils.data as data

# from src.model import SequenceLearner, NCP

# ncp = NCP(image_features.shape[1], output_size, units)
# ncp_model = ncp.get_ncp()
# learner = SequenceLearner(ncp_model, lr=0.01)
# trainer = pl.Trainer(
#     logger=pl.loggers.CSVLogger("log"),
#     max_epochs=5,
#     gradient_clip_val=1, #clip grad for training stabilizing
# )

Agent test

In [ ]:
import glob, os, sys
# sys.path.append(glob.glob('CARLA_SIM/PythonAPI/carla/'))
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent
# from agents.navigation.behavior_agent import BehaviorAgent

In [ ]:
simulator.spawn_car_with_camera()

In [ ]:
vehicle = simulator.get_vehicle()

In [ ]:
# agent = BehaviorAgent(vehicle, behavior='aggressive')
agent = BasicAgent(vehicle)
dest = simulator.spawn_points[50].location
agent.set_destination(dest)
agent.follow_speed_limits(False)
agent.set_target_speed(90)

In [ ]:
import time
while True:
    if agent.done():
        print("The target has been reached, stopping the simulation")
        break
    # if collided == True:
    #     vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))
    #     print("Collision detected. Abort")
    #     break

    vehicle.apply_control(agent.run_step())
    # time.sleep(0.5)

In [ ]:
simulator.destroy_all()